# Tutorial 1: Basic Poisson Equation Solving
## Mixed-Precision Multigrid Solvers - Step-by-Step Beginner Guide

Welcome to the Mixed-Precision Multigrid Solvers tutorial series! In this first tutorial, you'll learn how to solve the Poisson equation using our advanced multigrid solver.

### What You'll Learn:
- Basic concepts of the Poisson equation
- Setting up a computational grid
- Using the multigrid solver
- Visualizing and analyzing results
- Understanding convergence behavior

### Prerequisites:
- Basic Python knowledge
- Elementary understanding of partial differential equations
- NumPy and Matplotlib familiarity (helpful but not required)

## 1. Introduction to the Poisson Equation

The Poisson equation is a fundamental partial differential equation that appears in many areas of physics and engineering:

$$-\nabla^2 u = f$$

Where:
- $u(x,y)$ is the unknown solution function
- $f(x,y)$ is a known source term
- $\nabla^2$ is the Laplacian operator: $\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2}$

### Physical Interpretation:
- **Electrostatics**: $u$ = electric potential, $f$ = charge density
- **Heat conduction**: $u$ = temperature, $f$ = heat source
- **Fluid mechanics**: $u$ = stream function, $f$ = vorticity

In [ ]:
# Import necessary libraries
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import time

# Add the src directory to Python path
sys.path.insert(0, str(Path().parent / "src"))

# Import our multigrid solver components
from multigrid.core.grid import Grid
from multigrid.core.precision import PrecisionManager, PrecisionLevel
from multigrid.solvers.corrected_multigrid import CorrectedMultigridSolver

print("✅ All imports successful!")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib available: {plt.matplotlib.__version__}")

## 2. Setting Up Your First Problem

Let's start with a simple example: solving the Poisson equation on a unit square $[0,1] \times [0,1]$ with:
- **Source term**: $f(x,y) = 2\pi^2 \sin(\pi x) \sin(\pi y)$
- **Boundary conditions**: $u = 0$ on all boundaries (homogeneous Dirichlet)
- **Exact solution**: $u(x,y) = \sin(\pi x) \sin(\pi y)$ (we know this for validation!)

In [ ]:
# Step 1: Create a computational grid
print("Step 1: Creating computational grid")
print("="*40)

# Grid parameters
grid_size = 65  # 65x65 grid points
domain = (0.0, 1.0, 0.0, 1.0)  # [x_min, x_max, y_min, y_max]

# Create the grid
grid = Grid(nx=grid_size, ny=grid_size, domain=domain)

print(f"Grid created: {grid.nx}×{grid.ny} points")
print(f"Domain: [{domain[0]}, {domain[1]}] × [{domain[2]}, {domain[3]}]")
print(f"Grid spacing: hx = {grid.hx:.4f}, hy = {grid.hy:.4f}")
print(f"Total unknowns: {(grid.nx-2) * (grid.ny-2)} (excluding boundary points)")

In [ ]:
# Step 2: Set up the problem (source term and exact solution)
print("\nStep 2: Setting up the problem")
print("="*40)

# Create coordinate arrays
x = np.linspace(domain[0], domain[1], grid.nx)
y = np.linspace(domain[2], domain[3], grid.ny)
X, Y = np.meshgrid(x, y, indexing='ij')  # Create 2D coordinate grids

# Define the exact solution (for validation)
u_exact = np.sin(np.pi * X) * np.sin(np.pi * Y)

# Define the source term f(x,y) = 2π²sin(πx)sin(πy)
f_source = 2 * np.pi**2 * np.sin(np.pi * X) * np.sin(np.pi * Y)

# Apply homogeneous Dirichlet boundary conditions
# The boundary points are already zero for our exact solution
u_exact[0, :] = 0   # Left boundary
u_exact[-1, :] = 0  # Right boundary
u_exact[:, 0] = 0   # Bottom boundary
u_exact[:, -1] = 0  # Top boundary

f_source[0, :] = 0   # Set source to zero at boundaries
f_source[-1, :] = 0
f_source[:, 0] = 0
f_source[:, -1] = 0

print(f"Exact solution range: [{np.min(u_exact):.3f}, {np.max(u_exact):.3f}]")
print(f"Source term range: [{np.min(f_source):.3f}, {np.max(f_source):.3f}]")
print(f"Source term norm: {np.linalg.norm(f_source):.3f}")

In [ ]:
# Step 3: Visualize the problem setup
print("\nStep 3: Visualizing the problem setup")
print("="*40)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot the exact solution
im1 = ax1.contourf(X, Y, u_exact, levels=20, cmap='viridis')
ax1.set_title('Exact Solution: u(x,y) = sin(πx)sin(πy)')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
plt.colorbar(im1, ax=ax1)

# Plot the source term
im2 = ax2.contourf(X, Y, f_source, levels=20, cmap='RdBu')
ax2.set_title('Source Term: f(x,y) = 2π²sin(πx)sin(πy)')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
plt.colorbar(im2, ax=ax2)

plt.tight_layout()
plt.show()

print("💡 Tip: Notice how the source term has the same pattern as the solution")
print("   This is because -∇²u = f, so the Laplacian of our solution gives the source!")

## 3. Setting Up the Multigrid Solver

Now comes the exciting part - using our advanced multigrid solver! The multigrid method is one of the most efficient algorithms for solving large systems of linear equations arising from PDEs.

### Key Concepts:
- **Multigrid hierarchy**: Uses multiple grid levels (coarse and fine)
- **Mixed precision**: Automatically chooses between single and double precision
- **V-cycles**: Combines smoothing and coarse grid corrections

In [ ]:
# Step 4: Configure the precision manager
print("Step 4: Configuring precision management")
print("="*40)

# Create a precision manager that automatically switches between single and double precision
precision_manager = PrecisionManager(
    default_precision=PrecisionLevel.SINGLE,  # Start with single precision for speed
    adaptive=True,                           # Enable automatic switching
    convergence_threshold=1e-6,              # Switch to double if needed for convergence
    memory_threshold_gb=2.0                  # Memory constraint
)

print(f"Initial precision: {precision_manager.current_precision.value}")
print(f"Adaptive switching: {'Enabled' if precision_manager.adaptive else 'Disabled'}")
print(f"Convergence threshold: {precision_manager.convergence_threshold:.1e}")
print("\n💡 The solver will automatically switch to higher precision if needed!")

In [ ]:
# Step 5: Create and configure the multigrid solver
print("\nStep 5: Creating the multigrid solver")
print("="*40)

# Create the multigrid solver with optimal parameters
solver = CorrectedMultigridSolver(
    max_levels=4,                    # Use up to 4 grid levels
    max_iterations=30,               # Maximum iterations per V-cycle
    tolerance=1e-10,                 # Convergence tolerance
    cycle_type="V",                  # V-cycle (most common)
    pre_smooth_iterations=2,         # Smoothing before coarse correction
    post_smooth_iterations=2,        # Smoothing after coarse correction
    verbose=True                     # Show detailed output
)

print(f"Solver configuration:")
print(f"  Max levels: {solver.max_levels}")
print(f"  Max iterations: {solver.max_iterations}")
print(f"  Tolerance: {solver.tolerance:.1e}")
print(f"  Cycle type: {solver.cycle_type}")
print(f"  Smoothing: {solver.pre_smooth_iterations} pre + {solver.post_smooth_iterations} post")

# Display the multigrid hierarchy
print(f"\nMultigrid hierarchy:")
for i, g in enumerate(solver.grids):
    h = 1.0 / (g.nx - 1)
    dofs = (g.nx - 2) * (g.ny - 2)
    print(f"  Level {i}: {g.nx:3d}×{g.ny:3d} grid (h={h:.4f}, {dofs:5d} DOFs)")

## 4. Solving the Problem

Now let's solve our Poisson equation! We'll start with an initial guess of zero everywhere and let the multigrid solver work its magic.

In [ ]:
# Step 6: Solve the problem!
print("Step 6: Solving the Poisson equation")
print("="*50)

# Create initial guess (zero everywhere)
initial_guess = np.zeros_like(f_source)

print("Starting multigrid solver...")
print(f"Initial guess norm: {np.linalg.norm(initial_guess):.3f}")
print(f"Source term norm: {np.linalg.norm(f_source):.3f}")
print("\n" + "-"*50)

# Solve the system
start_time = time.time()
result = solver.solve(initial_guess, f_source, grid, precision_manager)
solve_time = time.time() - start_time

# Extract results
u_computed = result['solution']
converged = result['converged']
iterations = result['iterations']
final_residual = result['final_residual']
residual_history = result['residual_history']

print("-"*50)
print("🎉 SOLUTION COMPLETE!")
print("="*30)
print(f"Converged: {'✅ YES' if converged else '❌ NO'}")
print(f"Iterations: {iterations}")
print(f"Final residual: {final_residual:.2e}")
print(f"Solve time: {solve_time:.3f} seconds")
print(f"Time per iteration: {solve_time/iterations:.4f} seconds")

# Check precision usage
precision_stats = precision_manager.get_statistics()
print(f"\nPrecision usage:")
print(f"  Final precision: {precision_stats['current_precision']}")
print(f"  Precision switches: {len(precision_stats['precision_history']) - 1}")
if len(precision_stats['precision_history']) > 1:
    print(f"  Switch history: {' → '.join(precision_stats['precision_history'])}")

## 5. Analyzing the Results

Let's examine how well our solver performed by comparing with the exact solution and analyzing the convergence behavior.

In [ ]:
# Step 7: Compute and analyze errors
print("Step 7: Error analysis")
print("="*30)

# Compute errors
error = u_computed - u_exact
abs_error = np.abs(error)

# Error metrics (exclude boundary points)
interior_error = error[1:-1, 1:-1]
interior_exact = u_exact[1:-1, 1:-1]

l2_error = np.linalg.norm(interior_error)
max_error = np.max(np.abs(interior_error))
rms_error = np.sqrt(np.mean(interior_error**2))

# Relative errors
l2_norm_exact = np.linalg.norm(interior_exact)
relative_l2_error = l2_error / l2_norm_exact
relative_max_error = max_error / np.max(np.abs(interior_exact))

print(f"Error Analysis:")
print(f"  L2 error: {l2_error:.2e}")
print(f"  Max error: {max_error:.2e}")
print(f"  RMS error: {rms_error:.2e}")
print(f"  Relative L2 error: {relative_l2_error:.2e} ({relative_l2_error*100:.3f}%)")
print(f"  Relative max error: {relative_max_error:.2e} ({relative_max_error*100:.3f}%)")

# Grid spacing for context
h = 1.0 / (grid.nx - 1)
print(f"\nGrid spacing: h = {h:.4f}")
print(f"Expected O(h²) error: ~{h**2:.2e}")
print(f"Actual error vs expected: {l2_error / h**2:.1f}×")

# Quality assessment
if relative_l2_error < 1e-3:
    quality = "🏆 Excellent"
elif relative_l2_error < 1e-2:
    quality = "✅ Very Good"
elif relative_l2_error < 1e-1:
    quality = "👍 Good"
else:
    quality = "⚠️  Needs improvement"

print(f"\nSolution quality: {quality}")

In [ ]:
# Step 8: Visualize the results
print("\nStep 8: Visualizing the results")
print("="*35)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 12))

# 1. Computed solution
im1 = ax1.contourf(X, Y, u_computed, levels=20, cmap='viridis')
ax1.set_title('Computed Solution', fontsize=14, fontweight='bold')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
plt.colorbar(im1, ax=ax1)

# 2. Exact solution
im2 = ax2.contourf(X, Y, u_exact, levels=20, cmap='viridis')
ax2.set_title('Exact Solution', fontsize=14, fontweight='bold')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
plt.colorbar(im2, ax=ax2)

# 3. Error distribution
im3 = ax3.contourf(X, Y, abs_error, levels=20, cmap='Reds')
ax3.set_title(f'Absolute Error\n(Max: {max_error:.2e})', fontsize=14, fontweight='bold')
ax3.set_xlabel('x')
ax3.set_ylabel('y')
plt.colorbar(im3, ax=ax3)

# 4. Cross-section comparison
mid_j = grid.ny // 2
ax4.plot(x, u_computed[:, mid_j], 'b-', linewidth=2, label='Computed', alpha=0.8)
ax4.plot(x, u_exact[:, mid_j], 'r--', linewidth=2, label='Exact', alpha=0.8)
ax4.set_title(f'Cross-section at y = {y[mid_j]:.2f}', fontsize=14, fontweight='bold')
ax4.set_xlabel('x')
ax4.set_ylabel('u(x, y=0.5)')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.suptitle('Mixed-Precision Multigrid Solution Results', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 Observations:")
print("  • The computed and exact solutions should look nearly identical")
print("  • The error should be very small and concentrated away from boundaries")
print("  • The cross-section shows excellent agreement between computed and exact solutions")

## 6. Understanding Convergence Behavior

One of the most important aspects of iterative solvers is understanding how they converge. Let's analyze the convergence history of our multigrid solver.

In [ ]:
# Step 9: Analyze convergence behavior
print("Step 9: Convergence analysis")
print("="*35)

# Plot convergence history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 1. Residual history (logarithmic scale)
iterations_array = range(len(residual_history))
ax1.semilogy(iterations_array, residual_history, 'b-o', linewidth=2, markersize=4)

# Add theoretical convergence rates
if len(residual_history) > 1:
    # Linear convergence (typical for multigrid: factor ~0.1)
    linear_conv = [residual_history[0] * (0.1)**i for i in iterations_array]
    ax1.semilogy(iterations_array, linear_conv, 'g--', alpha=0.7, label='Theoretical (factor=0.1)')
    
    # Calculate actual convergence factor
    if len(residual_history) >= 3:
        factors = []
        for i in range(1, min(len(residual_history), 8)):  # Use first few iterations
            if residual_history[i-1] > 0:
                factor = residual_history[i] / residual_history[i-1]
                factors.append(factor)
        
        if factors:
            avg_factor = np.exp(np.mean(np.log(factors)))
            actual_conv = [residual_history[0] * (avg_factor)**i for i in iterations_array]
            ax1.semilogy(iterations_array, actual_conv, 'r:', alpha=0.7, 
                        label=f'Actual (factor≈{avg_factor:.3f})')
            
            print(f"Convergence Analysis:")
            print(f"  Average convergence factor: {avg_factor:.4f}")
            print(f"  Convergence rate: {-np.log(avg_factor):.2f}")
            
            if avg_factor < 0.1:
                conv_quality = "🏆 Excellent (optimal multigrid)"
            elif avg_factor < 0.3:
                conv_quality = "✅ Very good"
            elif avg_factor < 0.7:
                conv_quality = "👍 Good"
            else:
                conv_quality = "⚠️  Slow convergence"
            
            print(f"  Quality: {conv_quality}")

ax1.set_xlabel('Iteration')
ax1.set_ylabel('Residual Norm')
ax1.set_title('Convergence History', fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Add text box with convergence info
textstr = f'Initial: {residual_history[0]:.2e}\nFinal: {residual_history[-1]:.2e}\nReduction: {residual_history[-1]/residual_history[0]:.1e}'
props = dict(boxstyle='round', facecolor='lightblue', alpha=0.8)
ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=10,
        verticalalignment='top', bbox=props)

# 2. Convergence factor per iteration
if len(residual_history) > 1:
    conv_factors = []
    for i in range(1, len(residual_history)):
        if residual_history[i-1] > 0:
            factor = residual_history[i] / residual_history[i-1]
            conv_factors.append(factor)
    
    if conv_factors:
        ax2.plot(range(1, len(conv_factors)+1), conv_factors, 'ro-', linewidth=2, markersize=6)
        ax2.axhline(y=0.1, color='green', linestyle='--', alpha=0.7, label='Optimal (0.1)')
        ax2.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='No progress (1.0)')
        
        ax2.set_xlabel('Iteration')
        ax2.set_ylabel('Convergence Factor')
        ax2.set_title('Per-Iteration Convergence Factor', fontweight='bold')
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        ax2.set_ylim(0, min(2.0, max(conv_factors) * 1.1))

plt.tight_layout()
plt.show()

print(f"\n💡 Understanding the plots:")
print(f"  • Left: Residual should decrease exponentially (straight line on log scale)")
print(f"  • Right: Convergence factor <0.1 indicates excellent multigrid performance")
print(f"  • Multigrid achieves grid-independent convergence (same rate regardless of grid size!)")

## 7. Performance Analysis

Let's analyze the computational performance and understand the efficiency of our solver.

In [ ]:
# Step 10: Performance analysis
print("Step 10: Performance analysis")
print("="*35)

# Calculate key performance metrics
total_unknowns = (grid.nx - 2) * (grid.ny - 2)
work_units = iterations * total_unknowns  # Total computational work
work_per_unknown = work_units / total_unknowns

print(f"Performance Metrics:")
print(f"  Problem size: {total_unknowns:,} unknowns ({grid.nx}×{grid.ny} grid)")
print(f"  Total solve time: {solve_time:.4f} seconds")
print(f"  Time per iteration: {solve_time/iterations:.4f} seconds")
print(f"  Time per unknown per iteration: {solve_time/(iterations*total_unknowns)*1e6:.2f} μs")
print(f"  Iterations: {iterations}")
print(f"  Work units: {work_units:,} (iterations × unknowns)")

# Theoretical complexity analysis
theoretical_direct = total_unknowns**1.5  # Direct solver O(N^1.5) for 2D
theoretical_mg = total_unknowns * 10      # Multigrid O(N) with ~10 iterations

print(f"\nComplexity Comparison:")
print(f"  Our multigrid work: {work_units:,}")
print(f"  Theoretical MG work: {theoretical_mg:,}")
print(f"  Direct solver work: {theoretical_direct:,.0f}")
print(f"  Speedup vs direct: {theoretical_direct/work_units:.0f}×")

# Memory usage estimate
bytes_per_float = 8  # Double precision
arrays_needed = 4    # Solution, RHS, residual, auxiliary
memory_mb = (total_unknowns * arrays_needed * bytes_per_float) / (1024**2)

print(f"\nMemory Usage:")
print(f"  Estimated memory: {memory_mb:.1f} MB")
print(f"  Memory per unknown: {memory_mb*1024/total_unknowns:.1f} KB")

# Efficiency assessment
mg_efficiency = theoretical_mg / work_units
if mg_efficiency > 0.8:
    efficiency_rating = "🏆 Excellent efficiency"
elif mg_efficiency > 0.6:
    efficiency_rating = "✅ Good efficiency"
else:
    efficiency_rating = "⚠️  Could be more efficient"

print(f"\nEfficiency Assessment:")
print(f"  Multigrid efficiency: {mg_efficiency:.1%}")
print(f"  Rating: {efficiency_rating}")

# Create performance summary
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Computational work comparison
methods = ['Our\nMultigrid', 'Theoretical\nMultigrid', 'Direct\nSolver']
work_amounts = [work_units, theoretical_mg, theoretical_direct]
colors = ['blue', 'green', 'red']

bars = ax1.bar(methods, work_amounts, color=colors, alpha=0.7)
ax1.set_ylabel('Work Units (log scale)')
ax1.set_title('Computational Work Comparison', fontweight='bold')
ax1.set_yscale('log')

# Add value labels on bars
for bar, value in zip(bars, work_amounts):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{value:.1e}', ha='center', va='bottom')

# Time breakdown
time_breakdown = ['Setup', 'Solving', 'Analysis']
time_values = [0.001, solve_time, 0.002]  # Estimated times

ax2.pie(time_values, labels=time_breakdown, autopct='%1.1f%%', startangle=90)
ax2.set_title('Time Distribution', fontweight='bold')

plt.tight_layout()
plt.show()

## 8. Experimenting with Different Parameters

Now that you understand the basics, let's experiment with different solver parameters to see how they affect performance and accuracy.

In [ ]:
# Step 11: Parameter sensitivity analysis
print("Step 11: Parameter sensitivity analysis")
print("="*45)

# Test different tolerance values
print("Testing different tolerance values:")
print("-" * 40)

tolerances = [1e-6, 1e-8, 1e-10, 1e-12]
results_by_tol = []

for tol in tolerances:
    # Create solver with different tolerance
    test_solver = CorrectedMultigridSolver(
        max_levels=4,
        max_iterations=50,
        tolerance=tol,
        verbose=False  # Quiet for comparison
    )
    
    # Reset precision manager
    test_precision_manager = PrecisionManager(default_precision=PrecisionLevel.DOUBLE, adaptive=False)
    
    # Solve
    start_time = time.time()
    test_result = test_solver.solve(initial_guess, f_source, grid, test_precision_manager)
    test_time = time.time() - start_time
    
    # Compute error
    test_error = np.linalg.norm(test_result['solution'][1:-1, 1:-1] - u_exact[1:-1, 1:-1])
    
    results_by_tol.append({
        'tolerance': tol,
        'iterations': test_result['iterations'],
        'time': test_time,
        'error': test_error,
        'final_residual': test_result['final_residual'],
        'converged': test_result['converged']
    })
    
    status = "✅" if test_result['converged'] else "❌"
    print(f"  Tol {tol:.0e}: {test_result['iterations']:2d} iter, {test_time:.3f}s, error {test_error:.2e} {status}")

# Test different grid sizes (if memory allows)
print("\nTesting different grid sizes:")
print("-" * 40)

grid_sizes = [33, 65, 129] if solve_time < 0.1 else [33, 65]  # Adjust based on performance
results_by_size = []

for size in grid_sizes:
    # Create test problem
    test_grid = Grid(size, size, domain)
    x_test = np.linspace(0, 1, size)
    y_test = np.linspace(0, 1, size)
    X_test, Y_test = np.meshgrid(x_test, y_test, indexing='ij')
    
    u_exact_test = np.sin(np.pi * X_test) * np.sin(np.pi * Y_test)
    f_test = 2 * np.pi**2 * np.sin(np.pi * X_test) * np.sin(np.pi * Y_test)
    
    # Apply boundary conditions
    u_exact_test[0, :] = u_exact_test[-1, :] = u_exact_test[:, 0] = u_exact_test[:, -1] = 0
    f_test[0, :] = f_test[-1, :] = f_test[:, 0] = f_test[:, -1] = 0
    
    # Create solver
    test_solver = CorrectedMultigridSolver(
        max_levels=4, max_iterations=30, tolerance=1e-10, verbose=False
    )
    
    test_precision_manager = PrecisionManager(default_precision=PrecisionLevel.DOUBLE, adaptive=False)
    
    # Solve
    start_time = time.time()
    test_result = test_solver.solve(np.zeros_like(f_test), f_test, test_grid, test_precision_manager)
    test_time = time.time() - start_time
    
    # Compute error
    test_error = np.linalg.norm(test_result['solution'][1:-1, 1:-1] - u_exact_test[1:-1, 1:-1])
    h_test = 1.0 / (size - 1)
    
    results_by_size.append({
        'size': size,
        'h': h_test,
        'iterations': test_result['iterations'],
        'time': test_time,
        'error': test_error,
        'dofs': (size-2)**2
    })
    
    dofs = (size-2)**2
    print(f"  {size:3d}×{size:<3d}: {test_result['iterations']:2d} iter, {test_time:.3f}s, {dofs:5d} DOFs, error {test_error:.2e}")

print(f"\n💡 Key Observations:")
print(f"  • Tighter tolerance → more iterations but higher accuracy")
print(f"  • Grid size scaling → iterations should stay roughly constant (grid-independent convergence)")
print(f"  • Time should scale approximately linearly with problem size for optimal multigrid")

## 9. Summary and Next Steps

Congratulations! You've successfully solved your first Poisson equation using our mixed-precision multigrid solver. Let's summarize what we've learned and explore next steps.

In [ ]:
# Step 12: Summary and recommendations
print("🎉 TUTORIAL COMPLETE - SUMMARY")
print("="*50)

print("✅ What you've accomplished:")
print("  • Set up and solved a 2D Poisson equation")
print("  • Used advanced mixed-precision multigrid solver")
print("  • Analyzed solution accuracy and convergence behavior")
print("  • Explored parameter sensitivity")
print("  • Understood performance characteristics")

print(f"\n📊 Your results:")
print(f"  • Problem solved: {total_unknowns:,} unknowns in {solve_time:.3f} seconds")
print(f"  • Accuracy achieved: {relative_l2_error:.1e} relative L2 error")
print(f"  • Convergence: {iterations} iterations with {residual_history[-1]/residual_history[0]:.1e} reduction")
print(f"  • Precision: {precision_stats['current_precision']} precision used")

print(f"\n🚀 Next steps - Try these tutorials:")
print(f"  • Tutorial 2: Heat equation simulation (time-dependent problems)")
print(f"  • Tutorial 3: GPU acceleration demo (high-performance computing)")
print(f"  • Tutorial 4: Mixed-precision analysis (optimal precision selection)")
print(f"  • Tutorial 5: Custom boundary conditions (advanced physics)")
print(f"  • Tutorial 6: Performance tuning guide (optimization techniques)")

print(f"\n💡 Pro tips for your own problems:")
print(f"  • Start with coarser grids and work your way up")
print(f"  • Use adaptive precision for best accuracy/performance trade-off")
print(f"  • Monitor convergence - good multigrid should converge in <20 iterations")
print(f"  • Experiment with smoothing parameters for difficult problems")
print(f"  • Check your boundary conditions - they're often the source of issues!")

print(f"\n🎓 Theoretical concepts learned:")
print(f"  • Poisson equation: -∇²u = f")
print(f"  • Finite difference discretization")
print(f"  • Multigrid method with V-cycles")
print(f"  • Mixed-precision arithmetic")
print(f"  • Convergence analysis")

print(f"\nThank you for following this tutorial! 🙏")
print(f"Happy solving with mixed-precision multigrid! 🧮✨")

## Appendix: Quick Reference

Here's a quick reference for solving Poisson equations with our solver:

```python
# Basic setup
from multigrid.core.grid import Grid
from multigrid.core.precision import PrecisionManager, PrecisionLevel
from multigrid.solvers.corrected_multigrid import CorrectedMultigridSolver

# 1. Create grid
grid = Grid(nx=65, ny=65, domain=(0, 1, 0, 1))

# 2. Set up precision management
precision_manager = PrecisionManager(
    default_precision=PrecisionLevel.SINGLE,
    adaptive=True
)

# 3. Create solver
solver = CorrectedMultigridSolver(
    max_levels=4,
    max_iterations=30,
    tolerance=1e-10
)

# 4. Solve
result = solver.solve(initial_guess, rhs, grid, precision_manager)
solution = result['solution']
```

### Common Parameter Guidelines:
- **Grid size**: Start with 65×65, increase as needed
- **Max levels**: 4-6 levels work well for most problems
- **Tolerance**: 1e-10 for high accuracy, 1e-6 for engineering applications
- **Smoothing**: 2 pre + 2 post smoothing iterations (default)

### Troubleshooting:
- **Slow convergence**: Check boundary conditions, try different smoothing parameters
- **No convergence**: Reduce tolerance or increase max_iterations
- **Memory issues**: Enable adaptive precision or reduce grid size
- **Accuracy issues**: Use higher precision or refine the grid

Happy computing! 🚀